In [3]:
import pandas as pd
import math
import numpy as np

In [4]:
miami_weather = pd.read_csv('miami_weather_2013_2018.csv')

In [5]:
miami_weather.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [6]:
miami_weather.columns

Index(['CITY', 'DATE', 'apparentTemperatureHigh',
       'apparentTemperatureHighTime', 'apparentTemperatureLow',
       'apparentTemperatureLowTime', 'apparentTemperatureMax',
       'apparentTemperatureMaxTime', 'apparentTemperatureMin',
       'apparentTemperatureMinTime', 'cloudCover', 'dewPoint', 'humidity',
       'icon', 'moonPhase', 'ozone', 'precipIntensity', 'precipIntensityMax',
       'precipIntensityMaxTime', 'precipProbability', 'precipType', 'pressure',
       'summary', 'sunriseTime', 'sunsetTime', 'temperatureHigh',
       'temperatureHighTime', 'temperatureLow', 'temperatureLowTime',
       'temperatureMax', 'temperatureMaxTime', 'temperatureMin',
       'temperatureMinTime', 'time', 'uvIndex', 'uvIndexTime', 'visibility',
       'windBearing', 'windGust', 'windGustTime', 'windSpeed'],
      dtype='object')

In [40]:
miami_weather_subset = miami_weather[['DATE', 'CITY', 
                                      'temperatureHigh', 'temperatureLow', 
                                      'cloudCover', 'dewPoint', 'humidity', 
                                      'precipIntensity', 'precipProbability', 
                                      'precipType', 'pressure', 'uvIndex', 
                                      'visibility', 'windBearing', 
                                      'windGust', 'windSpeed']]

In [41]:
miami_weather_test = miami_weather_subset.replace({np.nan : None})

In [42]:
miami_weather_test.DATE.replace(list(miami_weather_test['DATE']), 
                                  [x.replace('T23:59:59', '') for x in miami_weather_test['DATE']], 
                                  inplace = True)
miami_weather_test.CITY.replace(['MIAMI'], ['Miami, FL'], inplace = True)

In [61]:
miami_weather_test.drop_duplicates(inplace = True)

In [62]:
miami_weather_test.shape

(372, 16)

# Convert data to tuples

In [63]:
miami_weather_tuples = [tuple(row) for row in miami_weather_test.values]

In [64]:
miami_weather_tuples[-5:-1]

[('2018-12-27',
  'Miami, FL',
  80.33,
  75.04,
  0.61,
  68.75,
  0.78,
  0.0001,
  0.3,
  'rain',
  1020.7,
  5,
  9.934,
  103,
  28.25,
  18.15),
 ('2018-12-28',
  'Miami, FL',
  81.7,
  73.2,
  0.8,
  72.65,
  0.85,
  0.0004,
  0.59,
  'rain',
  1020.5,
  4,
  9.904,
  128,
  18.85,
  13.27),
 ('2018-12-29',
  'Miami, FL',
  82.52,
  69.98,
  0.42,
  72.89,
  0.87,
  0.0,
  0.19,
  None,
  1021.3,
  4,
  9.953,
  132,
  11.6,
  8.15),
 ('2018-12-30',
  'Miami, FL',
  80.91,
  72.63,
  0.19,
  68.14,
  0.79,
  0.0,
  0.0,
  None,
  1021.1,
  5,
  9.829,
  108,
  12.04,
  8.0)]

# Load into MySQL

In [65]:
import mysql.connector
from mysql.connector import errorcode
import config

In [66]:
dbname = 'FLIGHTS'
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor()

In [67]:
insert_statement = """INSERT INTO FLIGHTS.WEATHER (date, city_name, max_temp, min_temp, cloud_cover, dew_point, humidity, precip_intens, precip_prob, precip_type, pressure, uv_index, visibility, windbearing, wind_gust, windspeed) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s)"""
cursor.executemany(insert_statement, miami_weather_tuples)
cnx.commit()

IntegrityError: 1062 (23000): Duplicate entry '2013-07-01' for key 'PRIMARY'

In [69]:
cursor.close()
cnx.close()